In [115]:
import cv2
import mediapipe as mp
import pyautogui
import math
from enum import IntEnum
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from google.protobuf.json_format import MessageToDict
import screen_brightness_control as sbcontrol

In [117]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [118]:
pip install opencv-contrib-python

In [119]:
!pip install opencv-contrib-python

In [120]:
import cv2
print(cv2.__version__)

4.10.0


In [121]:
pip install comtypes

Note: you may need to restart the kernel to use updated packages.


In [122]:
import comtypes

In [123]:
pip install pycaw

Note: you may need to restart the kernel to use updated packages.


In [124]:
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [125]:
pip install screen_brightness_control

Note: you may need to restart the kernel to use updated packages.


In [126]:
import mediapipe as mp
print(mp.__version__)

0.10.14


In [127]:
!pip install pyautogui

In [128]:
import mediapipe as mp
from mediapipe.python.solutions import hands as mp_hands

In [129]:
pyautogui.FAILSAFE = False
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [130]:
from mediapipe.python.solutions import drawing_utils

In [132]:
import tkinter as tk
import cv2 as cv
import numpy as np
import mediapipe as mp
import imutils 
from collections import deque
from threading import Thread
import pyautogui
from pynput import keyboard
# Class Controller
class Controller:
    def __init__(self, root):
        self.root = root
        self.root.title("Controller")

        # Control flags
        self.is_painting = False
        self.is_mouse_control = False
        self.is_keyboard_control = False

        # Create buttons
        self.start_paint_button = tk.Button(root, text="Start Virtual Paint", command=self.start_virtual_paint)
        self.start_paint_button.pack()

        self.exit_paint_button = tk.Button(root, text="Exit Virtual Paint", command=self.exit_virtual_paint)
        self.exit_paint_button.pack()

        self.start_mouse_button = tk.Button(root, text="Start Virtual Mouse", command=self.start_virtual_mouse)
        self.start_mouse_button.pack()

        self.exit_mouse_button = tk.Button(root, text="Exit Virtual Mouse", command=self.exit_virtual_mouse)
        self.exit_mouse_button.pack()

        self.start_keyboard_button = tk.Button(root, text="Start Virtual Keyboard", command=self.start_virtual_keyboard)
        self.start_keyboard_button.pack()

        self.exit_keyboard_button = tk.Button(root, text="Exit Virtual Keyboard", command=self.exit_virtual_keyboard)
        self.exit_keyboard_button.pack()

        self.exit_button = tk.Button(root, text="Exit Application", command=self.exit_application)
        self.exit_button.pack()

        # Threads to manage functionality no thread is currently active
        self.paint_thread = None
        self.mouse_thread = None
        self.keyboard_thread = None
        self.keyboard_listener = None

    def start_virtual_paint(self):
        if not self.is_painting:
            self.is_painting = True
            self.paint_thread = Thread(target=self.virtual_paint)
            self.paint_thread.start()

    def exit_virtual_paint(self):
        self.is_painting = False
        if self.paint_thread:
            self.paint_thread.join()
        cv2.destroyAllWindows()

    def start_virtual_mouse(self):
        if not self.is_mouse_control:
            self.is_mouse_control = True
            self.mouse_thread = Thread(target=self.virtual_mouse)
            self.mouse_thread.start()

    def exit_virtual_mouse(self):
        self.is_mouse_control = False
        if self.mouse_thread:
            self.mouse_thread.join()

    def start_virtual_keyboard(self):
        if not self.is_keyboard_control:
            self.is_keyboard_control = True
            self.keyboard_listener = keyboard.Listener(on_press=self.on_key_press)
            self.keyboard_listener.start()
            self.keyboard_thread = Thread(target=self.virtual_keyboard)
            self.keyboard_thread.start()

    def exit_virtual_keyboard(self):
        self.is_keyboard_control = False
        if self.keyboard_listener:
            self.keyboard_listener.stop()
        if self.keyboard_thread:
            self.keyboard_thread.join()

    def exit_application(self):
        self.is_painting = False
        self.is_mouse_control = False
        self.is_keyboard_control = False
        
        if self.paint_thread:
            self.paint_thread.join()
        if self.mouse_thread:
            self.mouse_thread.join()
        if self.keyboard_thread:
            self.keyboard_thread.join()
        if self.keyboard_listener:
            self.keyboard_listener.stop()
        
        self.root.destroy()
        cv2.destroyAllWindows()

    def virtual_paint(self):
        bpoints = [deque(maxlen=1024)]
        gpoints = [deque(maxlen=1024)]
        rpoints = [deque(maxlen=1024)]
        ypoints = [deque(maxlen=1024)]
        #These deques are used to store the points for drawing in different colors

        blue_index = 0
        green_index = 0
        red_index = 0
        yellow_index = 0
        #defining kernel and colour values and colour index

        kernel = np.ones((5,5),np.uint8)
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
        colorIndex = 0

        paintWindow = np.zeros((471,636,3)) + 255
        paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
        paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), (255,0,0), 2)
        paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), (0,255,0), 2)
        paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), (0,0,255), 2)
        paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), (0,255,255), 2)

        cv2.putText(paintWindow, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(paintWindow, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(paintWindow, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(paintWindow, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(paintWindow, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)
        #Initializes MediaPipe's hand detection module to track one hand with a minimum detection confidence of 70%.

        mpHands = mp.solutions.hands
        hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mpDraw = mp.solutions.drawing_utils

        cap = cv2.VideoCapture(0)
        ret = True

        while ret and self.is_painting:
            ret, frame = cap.read()

            x, y, c = frame.shape
            frame = cv2.flip(frame, 1)
            framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            frame = cv2.rectangle(frame, (40,1), (140,65), (0,0,0), 2)
            frame = cv2.rectangle(frame, (160,1), (255,65), (255,0,0), 2)
            frame = cv2.rectangle(frame, (275,1), (370,65), (0,255,0), 2)
            frame = cv2.rectangle(frame, (390,1), (485,65), (0,0,255), 2)
            frame = cv2.rectangle(frame, (505,1), (600,65), (0,255,255), 2)
            cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            result = hands.process(framergb)

            if result.multi_hand_landmarks:
                landmarks = []
                for handslms in result.multi_hand_landmarks:
                    for lm in handslms.landmark:
                        lmx = int(lm.x * 640)
                        lmy = int(lm.y * 480)
                        landmarks.append([lmx, lmy])

                    mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

                fore_finger = (landmarks[8][0], landmarks[8][1])
                center = fore_finger
                thumb = (landmarks[4][0], landmarks[4][1])
                cv2.circle(frame, center, 3, (0, 255, 0), -1)

                if thumb[1] - center[1] < 30:
                    bpoints.append(deque(maxlen=512))
                    blue_index += 1
                    gpoints.append(deque(maxlen=512))
                    green_index += 1
                    rpoints.append(deque(maxlen=512))
                    red_index += 1
                    ypoints.append(deque(maxlen=512))
                    yellow_index += 1

                elif center[1] <= 65:
                    if 40 <= center[0] <= 140:  # Clear Button
                        bpoints = [deque(maxlen=512)]
                        gpoints = [deque(maxlen=512)]
                        rpoints = [deque(maxlen=512)]
                        ypoints = [deque(maxlen=512)]

                        blue_index = 0
                        green_index = 0
                        red_index = 0
                        yellow_index = 0

                        paintWindow[67:, :, :] = 255
                    elif 160 <= center[0] <= 255:
                        colorIndex = 0  # Blue
                    elif 275 <= center[0] <= 370:
                        colorIndex = 1  # Green
                    elif 390 <= center[0] <= 485:
                        colorIndex = 2  # Red
                    elif 505 <= center[0] <= 600:
                        colorIndex = 3  # Yellow
                else:
                    if colorIndex == 0:
                        bpoints[blue_index].appendleft(center)
                    elif colorIndex == 1:
                        gpoints[green_index].appendleft(center)
                    elif colorIndex == 2:
                        rpoints[red_index].appendleft(center)
                    elif colorIndex == 3:
                        ypoints[yellow_index].appendleft(center)
            else:
                bpoints.append(deque(maxlen=512))
                blue_index += 1
                gpoints.append(deque(maxlen=512))
                green_index += 1
                rpoints.append(deque(maxlen=512))
                red_index += 1
                ypoints.append(deque(maxlen=512))
                yellow_index += 1

            points = [bpoints, gpoints, rpoints, ypoints]
            for i in range(len(points)):
                for j in range(len(points[i])):
                    for k in range(1, len(points[i][j])):
                        if points[i][j][k - 1] is None or points[i][j][k] is None:
                            continue
                        cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                        cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

            cv2.imshow("Output", frame)
            cv2.imshow("Paint", paintWindow)

            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    def virtual_mouse(self):
        #gesture enumeration
        class Gest(IntEnum):
            FIST = 0
            PINKY = 1
            RING = 2
            MID = 4
            LAST3 = 7
            INDEX = 8
            FIRST2 = 12
            LAST4 = 15
            THUMB = 16    
            PALM = 31
            # normal mappings kakundha extra mappings
            V_GEST = 33
            TWO_FINGER_CLOSED = 34
            PINCH_MAJOR = 35
            PINCH_MINOR = 36
            # two hands use cheykandi
        class HLabel(IntEnum):
            MINOR = 0
            MAJOR = 1
            #hand recognition class
            # Mediapipe use chesi ey symbol vadamo guess chestundhi
        class HandRecog:
            def __init__(self, hand_label):
                self.finger = 0
                self.ori_gesture = Gest.PALM
                self.prev_gesture = Gest.PALM
                self.frame_count = 0
                self.hand_result = None
                self.hand_label = hand_label
            def update_hand_result(self, hand_result):
                self.hand_result = hand_result
            def get_signed_dist(self, point):
                sign = -1
                if self.hand_result.landmark[point[0]].y < self.hand_result.landmark[point[1]].y:
                    sign = 1
                dist = (self.hand_result.landmark[point[0]].x - self.hand_result.landmark[point[1]].x)**2
                dist += (self.hand_result.landmark[point[0]].y - self.hand_result.landmark[point[1]].y)**2
                dist = math.sqrt(dist)
                return dist*sign
            
            def get_dist(self, point):
                dist = (self.hand_result.landmark[point[0]].x - self.hand_result.landmark[point[1]].x)**2
                dist += (self.hand_result.landmark[point[0]].y - self.hand_result.landmark[point[1]].y)**2
                dist = math.sqrt(dist)
                return dist
            
            def get_dz(self,point):
                return abs(self.hand_result.landmark[point[0]].z - self.hand_result.landmark[point[1]].z)
                # oka vele  finger open lo untey 1 ledha (else) 0
            def set_finger_state(self):
                if self.hand_result is None:
                    return
        
                points = [[8,5,0],[12,9,0],[16,13,0],[20,17,0]]
                self.finger = 0
                self.finger = self.finger | 0 
                for idx,point in enumerate(points):
                    
                    dist = self.get_signed_dist(point[:2])
                    dist2 = self.get_signed_dist(point[1:])
                    
                    try:
                        ratio = round(dist/dist2,1)
                    except:
                        ratio = round(dist1/0.01,1)
        
                    self.finger = self.finger << 1
                    if ratio > 0.5:
                        self.finger = self.finger | 1
            # Fluctations untey detect chestundhi
            def get_gesture(self):
                if self.hand_result is None:
                    return Gest.PALM
        
                current_gesture = Gest.PALM
                if self.finger in [Gest.LAST3, Gest.LAST4] and self.get_dist([8,4]) < 0.05:
                    if self.hand_label == HLabel.MINOR:
                        current_gesture = Gest.PINCH_MINOR
                    else:
                        current_gesture = Gest.PINCH_MAJOR
        
                elif Gest.FIRST2 == self.finger:
                    point = [[8,12],[5,9]]
                    dist1 = self.get_dist(point[0])
                    dist2 = self.get_dist(point[1])
                    ratio = dist1/dist2
                    if ratio > 1.7:
                        current_gesture = Gest.V_GEST
                    else:
                        if self.get_dz([8,12]) < 0.1:
                            current_gesture = Gest.TWO_FINGER_CLOSED
                        else:
                            current_gesture = Gest.MID
                    
                else:
                    current_gesture = self.finger
                
                if current_gesture == self.prev_gesture:
                    self.frame_count += 1
                else:
                    self.frame_count = 0
        
                self.prev_gesture = current_gesture
        
                if self.frame_count > 4:
                    self.ori_gesture = current_gesture
                return self.ori_gesture 
                # gesture ni detect chesina taruvata a gesture ki function ivalo set chestundhi
        class Controller:
            tx_old = 0
            ty_old = 0
            trial = True
            flag = False
            grabflag = False
            pinchmajorflag = False
            pinchminorflag = False
            pinchstartxcoord = None
            pinchstartycoord = None
            pinchdirectionflag = None
            prevpinchlv = 0
            pinchlv = 0
            framecount = 0
            prev_hand = None
            pinch_threshold = 0.3
            
            
            def getpinchylv(hand_result):
                dist = round((Controller.pinchstartycoord - hand_result.landmark[8].y)*10,1)
                return dist
        
        
            def getpinchxlv(hand_result):
                dist = round((hand_result.landmark[8].x - Controller.pinchstartxcoord)*10,1)
                return dist
            
            
            def changesystembrightness():
                currentBrightnessLv = sbcontrol.get_brightness()/100.0
                currentBrightnessLv += Controller.pinchlv/50.0
                if currentBrightnessLv > 1.0:
                    currentBrightnessLv = 1.0
                elif currentBrightnessLv < 0.0:
                    currentBrightnessLv = 0.0       
                sbcontrol.fade_brightness(int(100*currentBrightnessLv) , start = sbcontrol.get_brightness())
            
            
            def changesystemvolume():
                devices = AudioUtilities.GetSpeakers()
                interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
                volume = cast(interface, POINTER(IAudioEndpointVolume))
                currentVolumeLv = volume.GetMasterVolumeLevelScalar()
                currentVolumeLv += Controller.pinchlv/50.0
                if currentVolumeLv > 1.0:
                    currentVolumeLv = 1.0
                elif currentVolumeLv < 0.0:
                    currentVolumeLv = 0.0
                volume.SetMasterVolumeLevelScalar(currentVolumeLv, None)
            def start_virtual_mouse(self):
                if not self.is_mouse_control:
                    self.is_mouse_control = True
                    self.mouse_thread = Thread(target=self.virtual_mouse)
                    self.mouse_thread.start()
            
            
            def scrollVertical():
                pyautogui.scroll(120 if Controller.pinchlv > 0.0 else -120)
        
            
            def scrollHorizontal():
                pyautogui.keyDown('shift')
                pyautogui.keyDown('ctrl')
                pyautogui.scroll(-120 if Controller.pinchlv > 0.0 else 120)
                pyautogui.keyUp('ctrl')
                pyautogui.keyUp('shift')
        
            # cursor ey position lo undho find chestundhi
            def get_position(hand_result):
                point = 9
                position = [hand_result.landmark[point].x ,hand_result.landmark[point].y]
                sx, sy = pyautogui.size()
                x_old, y_old = pyautogui.position()
                x = int(position[0]*sx)
                y = int(position[1]*sy)
                if Controller.prev_hand is None:
                    Controller.prev_hand = x, y
                delta_x = x - Controller.prev_hand[0]
                delta_y = y - Controller.prev_hand[1]
        
                distsq = delta_x**2 + delta_y**2
                ratio = 1
                Controller.prev_hand = [x, y]
        
                if distsq <= 25:
                    ratio = 0
                elif distsq <= 900:
                    ratio = 0.07 * (distsq ** (1/2))
                else:
                    ratio = 2.1
                x, y = x_old + delta_x * ratio, y_old + delta_y * ratio
                return (x, y)
            
            def handle_controls(gest_name, hand_result):
                if gest_name == Gest.PINCH_MAJOR:
                    Controller.pinchmajorflag = True
                    Controller.pinchminorflag = False
                    Controller.pinchstartxcoord = hand_result.landmark[8].x
                    Controller.pinchstartycoord = hand_result.landmark[8].y
                    Controller.pinchlv = 0
                elif gest_name == Gest.PINCH_MINOR:
                    Controller.pinchminorflag = True
                    Controller.pinchmajorflag = False
                    Controller.pinchstartxcoord = hand_result.landmark[8].x
                    Controller.pinchstartycoord = hand_result.landmark[8].y
                    Controller.pinchlv = 0
                elif gest_name == Gest.PALM:
                    Controller.pinchminorflag = False
                    Controller.pinchmajorflag = False
                    Controller.pinchstartxcoord = None
                    Controller.pinchstartycoord = None
                elif gest_name == Gest.MID:
                    if Controller.pinchminorflag:
                        Controller.pinchlv = Controller.getpinchylv(hand_result)
                        Controller.changesystembrightness()
                    elif Controller.pinchmajorflag:
                        Controller.pinchlv = Controller.getpinchylv(hand_result)
                        Controller.changesystemvolume()
                    else:
                        position = Controller.get_position(hand_result)
                        pyautogui.moveTo(position[0], position[1])
                elif gest_name == Gest.V_GEST:
                    position = Controller.get_position(hand_result)
                    pyautogui.moveTo(position[0], position[1])
                elif gest_name == Gest.TWO_FINGER_CLOSED:
                    if Controller.trial:
                        Controller.trial = False
                        Controller.scrollHorizontal()
                else:
                    position = Controller.get_position(hand_result)
                    pyautogui.moveTo(position[0], position[1])
            def handle_controls(gesture, hand_result):        
                x,y = None,None
                if gesture != Gest.PALM :
                    x,y = Controller.get_position(hand_result)
                
                
                if gesture != Gest.FIST and Controller.grabflag:
                    Controller.grabflag = False
                    pyautogui.mouseUp(button = "left")
        
                if gesture != Gest.PINCH_MAJOR and Controller.pinchmajorflag:
                    Controller.pinchmajorflag = False
        
                if gesture != Gest.PINCH_MINOR and Controller.pinchminorflag:
                    Controller.pinchminorflag = False
        
                
                if gesture == Gest.V_GEST:
                    Controller.flag = True
                    pyautogui.moveTo(x, y, duration = 0.1)
        
                elif gesture == Gest.FIST:
                    if not Controller.grabflag : 
                        Controller.grabflag = True
                        pyautogui.mouseDown(button = "left")
                    pyautogui.moveTo(x, y, duration = 0.1)
        
                elif gesture == Gest.MID and Controller.flag:
                    pyautogui.click()
                    Controller.flag = False
        
                elif gesture == Gest.INDEX and Controller.flag:
                    pyautogui.click(button='right')
                    Controller.flag = False
        
                elif gesture == Gest.TWO_FINGER_CLOSED and Controller.flag:
                    pyautogui.doubleClick()
                    Controller.flag = False
        
                elif gesture == Gest.PINCH_MINOR:
                    if Controller.pinchminorflag == False:
                        Controller.pinch_control_init(hand_result)
                        Controller.pinchminorflag = True
                    Controller.pinch_control(hand_result,Controller.scrollHorizontal, Controller.scrollVertical)
                
                elif gesture == Gest.PINCH_MAJOR:
                    if Controller.pinchmajorflag == False:
                        Controller.pinch_control_init(hand_result)
                        Controller.pinchmajorflag = True
                    Controller.pinch_control(hand_result,Controller.changesystembrightness, Controller.changesystemvolume)
        class GestureController:
            gc_mode = 0
            cap = None
            CAM_HEIGHT = None
            CAM_WIDTH = None
            hr_major = None 
            hr_minor = None 
            dom_hand = True
        
            def __init__(self):
                GestureController.gc_mode = 1
                GestureController.cap = cv2.VideoCapture(0)
                GestureController.CAM_HEIGHT = GestureController.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
                GestureController.CAM_WIDTH = GestureController.cap.get(cv2.CAP_PROP_FRAME_WIDTH)
            
            
            def classify_hands(results):
                left , right = None, None
                try:
                    handedness_dict = MessageToDict(results.multi_handedness[0])
                    if handedness_dict['classification'][0]['label'] == 'Right':
                        right = results.multi_hand_landmarks[0]
                    else:
                        left = results.multi_hand_landmarks[0]
                except:
                    pass
        
                try:
                    handedness_dict = MessageToDict(results.multi_handedness[1])
                    if handedness_dict['classification'][0]['label'] == 'Right':
                        right = results.multi_hand_landmarks[1]
                    else:
                        left = results.multi_hand_landmarks[1]
                except:
                    pass
                
                if GestureController.dom_hand:
                    GestureController.hr_major = right
                    GestureController.hr_minor = left
                else:
                    GestureController.hr_major = left
                    GestureController.hr_minor = right
        
            def start(self):
                handmajor = HandRecog(HLabel.MAJOR)
                handminor = HandRecog(HLabel.MINOR)
                with mp_hands.Hands(max_num_hands = 2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:       
                    while GestureController.gc_mode and self.is_mouse_control :
                        success, image = GestureController.cap.read()
                        if not success:
                            print("Ignoring empty camera frame.")
                            continue
                        
                        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                        image.flags.writeable = False
                        results = hands.process(image)
                        
                        image.flags.writeable = True
                        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
                        if results.multi_hand_landmarks:                   
                            GestureController.classify_hands(results)
                            handmajor.update_hand_result(GestureController.hr_major)
                            handminor.update_hand_result(GestureController.hr_minor)
        
                            handmajor.set_finger_state()
                            handminor.set_finger_state()
                            gest_name = handminor.get_gesture()
        
                            if gest_name == Gest.PINCH_MINOR:
                                Controller.handle_controls(gest_name, handminor.hand_result)
                            else:
                                gest_name = handmajor.get_gesture()
                                Controller.handle_controls(gest_name, handmajor.hand_result)
                            
                            for hand_landmarks in results.multi_hand_landmarks:
                                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                        else:
                            Controller.prev_hand = None
                        cv2.imshow("Gesturecontroller", image)
                        if cv2.waitKey(5) & 0xFF == ord('q'):
                            break
                Gesturecontroller.cap.release()
                cv2.destroyAllWindows()

                    # Capture frame and process mouse gestures here
                    

    def virtual_keyboard(self):
        # Initialize variables for virtual keyboard control
        #camera setup
        cam = cv.VideoCapture(0)
        arr = []
        #keyboard layout
        nums = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
        row1 = ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"]
        row2 = ["A", "S", "D", "F", "G", "H", "J", "K", "L"]
        row3 = ["Z", "X", "C", "V", "B", "N", "M"]
        row4 = ["space", "enter", "backspace", "shift"]
        row5 = ["left", "up", "down", "right"]
        row6 = ["volumeup", "volumedown", "volumemute"]
        x, y = 10, 20
        #creating key data to keyboard
        for i in range(10):
            data = {"x": x, "y": y, "w": 100, "h": 80, "value": nums[i]}
            arr.append(data)
            x += 100
        
        y, x = 100, 10
        for i in range(10):
            data = {"x": x, "y": y, "w": 100, "h": 80, "value": row1[i]}
            arr.append(data)
            x += 100
        
        x, y = 110, 180
        for i in range(9):
            data = {"x": x, "y": y, "w": 100, "h": 80, "value": row2[i]}
            arr.append(data)
            x += 100
        
        x, y = 210, 260
        for i in range(7):
            data = {"x": x, "y": y, "w": 100, "h": 80, "value": row3[i]}
            arr.append(data)
            x += 100
        
        x, y = 110, 340
        for i in range(4):
            width = 200 if i < 2 else 250 if i == 2 else 200
            data = {"x": x, "y": y, "w": width, "h": 80, "value": row4[i]}
            arr.append(data)
            x += width + 10
        
        x, y = 110, 420
        for i in range(4):
            data = {"x": x, "y": y, "w": 200, "h": 80, "value": row5[i]}
            arr.append(data)
            x += 200
        
        x, y = 10, 500
        for i in range(3):
            data = {"x": x, "y": y, "w": 200, "h": 80, "value": row6[i]}
            arr.append(data)
            x += 200
        
        json_string = json.dumps(arr)
        json_data = json.loads(json_string)
        #The main loop runs while the virtual keyboard is active
        while self.is_keyboard_control:
            ret, img = cam.read()
            img = cv.GaussianBlur(img, (5, 5), 0)
            img = imutils.resize(img, width=1030, height=700)
            height, width = img.shape[:2]
            x, y = 10, 20
            
            for i in range(10):
                cv.rectangle(img, (x, y), (x + 100, y + 80), (0, 255, 255), 2)
                cv.putText(img, nums[i], (x + 50, y + 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA, False)
                x += 100
            
            y, x = 100, 10
            for i in range(10):
                cv.rectangle(img, (x, y), (x + 100, y + 80), (0, 255, 255), 2)
                cv.putText(img, row1[i], (x + 50, y + 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA, False)
                x += 100
            
            x, y = 110, 180
            for i in range(9):
                cv.rectangle(img, (x, y), (x + 100, y + 80), (0, 255, 255), 2)
                cv.putText(img, row2[i], (x + 50, y + 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA, False)
                x += 100
            
            x, y = 210, 260
            for i in range(7):
                cv.rectangle(img, (x, y), (x + 100, y + 80), (0, 255, 255), 2)
                cv.putText(img, row3[i], (x + 50, y + 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA, False)
                x += 100
            
            x, y = 110, 340
            for i in range(4):
                width = 200 if i < 2 else 250 if i == 2 else 200
                cv.rectangle(img, (x, y), (x + width, y + 80), (0, 255, 255), 2)
                cv.putText(img, row4[i], (x + 70, y + 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA, False)
                x += width + 10
            
            x, y = 110, 420
            for i in range(4):
                cv.rectangle(img, (x, y), (x + 200, y + 80), (0, 255, 255), 2)
                cv.putText(img, row5[i], (x + 100, y + 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA, False)
                x += 200
            
            x, y = 10, 500
            for i in range(3):
                cv.rectangle(img, (x, y), (x + 200, y + 80), (0, 255, 255), 2)
                label = ["V+", "V-", "Vmute"][i]
                cv.putText(img, label, (x + 60, y + 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA, False)
                x += 200
            #hue,saturation and value and  Bgr
            hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
            mask = cv.inRange(hsv_img, np.array([65, 60, 60]), np.array([80, 255, 255]))
            mask_open = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((5, 5)))
            mask_close = cv.morphologyEx(mask_open, cv.MORPH_CLOSE, np.ones((20, 20)))
            mask_final = mask_close
            conts, _ = cv.findContours(mask_final.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            #Contours are detected in the processed mask to identify shapes corresponding to hand gestures.
            #calculate bounding rectangle
            if len(conts) == 1:
                x, y, w, h = cv.boundingRect(conts[0])
                cx = round(x + w / 2) 
                cy = round(y + h / 2)
                cv.circle(img, (cx, cy), 20, (0, 0, 255), 2)
                word = ""
                for i in range(len(json_data)):
                    if (cx >= int(json_data[i]["x"]) and cx <= int(json_data[i]["x"]) + int(json_data[i]["w"]) and
                        cy >= int(json_data[i]["y"]) and cy <= int(json_data[i]["y"]) + int(json_data[i]["h"])):
                        word = json_data[i]["value"]
                
                if word:
                    pyautogui.press(word)
                    #simulate key press
            
            
            cv.imshow("cam", img)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break

        cam.release()
        cv.destroyAllWindows()

        
            # Capture frame and process keyboard gestures here

    def on_key_press(self, key):
        try:
            print(f"Key {key.char} pressed")
            # Simulate key press
        except AttributeError:
            print(f"Special key {key} pressed")

if __name__ == "__main__":
    root = tk.Tk()
    controller = Controller(root)
    root.mainloop()